DATA-SPECIFIC INFORMATION FOR: 
eaglei_outages_2023.csv 1.2 GB

1. Number of variables: 5

2. Number of cases/rows: 26101052

3. Variable List: 
 - Fips_code: The FIPS code of the county in which the power outages occurred, for example “12011”
 - County: The county name in which the power outages occurred spelled out in text, for example “Broward”
 - State: The state in which the power outage occurred, spelled out in full in text format. For example, “Florida”
 - Sum: The total number of customers without power for that county at that timestamp. This number is always an integer. Entries with 0 customers without power were not included in this dataset. Note that the number of “customers” does not necessarily equate to the number of people affected, as a “customer” reported by a utility could be one meter, one building, etc. Outages are collected at various different levels of resolution (county, point, zip code, and polygon) and are aggregated to county for consistent reporting.
 - Run_start_time: Date and timestamp provided in GMT in the format “MM/DD/YY 00:00”. EAGLE-I collects power outage information from all covered utilities at 15-minute intervals, and this timestamp marks the beginning of the collection run.


In [19]:
import os, sys

import pandas as pd
import numpy as np

import geopandas as gpd


import plotly.express as px
import json

import matplotlib.pyplot as plt

import time

import glob

import imageio.v2 as imageio  # For creating a GIF


In [90]:
directory = '/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/outage_data/EAGLE-I/'
file = 'outage_data_2023.csv'

data = pd.read_csv(os.path.join(directory,file))
data.head()

data['datetime'] = pd.to_datetime(data['run_start_time'], errors='coerce')  # Handle any invalid dates

# Define start and end dates
start_date = '2023-03-31'
end_date = '2023-04-04'

# Filter rows that fall within the date range
filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]




In [12]:
pd.unique(filtered_data['state'])

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Puerto Rico', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'United States Virgin Islands', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming',
       'Rhode Island'], dtype=object)

In [59]:

# Load the US County GeoJSON file
geojson_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
geojson = json.loads(pd.read_json(geojson_url).to_json())
counties_gdf = gpd.read_file(geojson_url)

# geojson_file = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/location_data/counties.geojson"
# geojson = json.loads(pd.read_json(geojson_url).to_json())


In [60]:
counties_gdf.head()

,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,01001,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,01009,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,01017,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,01021,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,01033,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."


In [61]:
counties_gdf['fips_code'] = pd.to_numeric(counties_gdf['id'], downcast='integer', errors='coerce')


In [62]:
counties_gdf.head()

,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry,fips_code
0,01001,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",1001
1,01009,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",1009
2,01017,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413 32.87053, -85.12342 32.772...",1017
3,01021,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",1021
4,01033,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.13999 34.58170, -88.13925 34.587...",1033


In [5]:
# for key, feature in geojson['features'].items():  # Iterate through dictionary keys and values
#     if "properties" in feature:  # Ensure "properties" exists
#         state_fp = feature["properties"].get("STATEFP", "")
#         county_fp = feature["properties"].get("COUNTYFP", "")
#         if state_fp and county_fp:
#             full_fips = f"{state_fp}{county_fp}"
#             feature["properties"]["fips"] = full_fips  # Add "fips" field

In [15]:
# geojson['features']['0']


In [91]:
filtered_data

,fips_code,county,state,sum,run_start_time,datetime
5654823,1009,Blount,Alabama,4,2023-03-31 00:00:00,2023-03-31
5654824,1051,Elmore,Alabama,4,2023-03-31 00:00:00,2023-03-31
5654825,1055,Etowah,Alabama,6,2023-03-31 00:00:00,2023-03-31
5654826,1061,Geneva,Alabama,4,2023-03-31 00:00:00,2023-03-31
5654827,1069,Houston,Alabama,4,2023-03-31 00:00:00,2023-03-31
...,...,...,...,...,...,...
6025685,55117,Sheboygan,Wisconsin,2,2023-04-04 00:00:00,2023-04-04
6025686,55121,Trempealeau,Wisconsin,1,2023-04-04 00:00:00,2023-04-04
6025687,55125,Vilas,Wisconsin,2,2023-04-04 00:00:00,2023-04-04
6025688,55139,Winnebago,Wisconsin,1,2023-04-04 00:00:00,2023-04-04


In [55]:
# Define the target datetime
target_date = pd.to_datetime("2023-03-31 00:00:00")

# Filter rows for the given datetime
time_instance_df = filtered_data[filtered_data["datetime"] == target_date]

merged_time_instance = counties_gdf.merge(time_instance_df, on="fips_code", how="left")

merged_time_instance = merged_time_instance.dropna(subset=['run_start_time'])
cols_to_fill = ['run_start_time', 'datetime']
merged_time_instance[cols_to_fill] = merged_time_instance[cols_to_fill].ffill()

# merged_with_outages["Su
merged_time_instance["sum"].fillna(0, inplace=True)

# # Plot the map
# fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# merged_time_instance.boundary.plot(ax=ax, linewidth=0.5)  # Plot county boundaries
# merged_time_instance.plot(column="sum", ax=ax, cmap="OrRd", legend=True, edgecolor="black")

# ax.set_title("Power Grid Outages by County")
# plt.show()


In [56]:
merged_time_instance

,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry,fips_code,county,state,sum,run_start_time,datetime
8,01079,0500000US01079,01,079,Lawrence,County,690.678,"POLYGON ((-87.10507 34.68604, -87.10591 34.587...",1079,Lawrence,Alabama,9.0,2023-03-26 00:00:00,2023-03-26
12,01121,0500000US01121,01,121,Talladega,County,736.775,"POLYGON ((-86.17437 33.10439, -86.22627 33.104...",1121,Talladega,Alabama,4.0,2023-03-26 00:00:00,2023-03-26
22,05145,0500000US05145,05,145,White,County,1035.075,"POLYGON ((-91.46511 35.08945, -91.58401 35.091...",5145,White,Arkansas,5.0,2023-03-26 00:00:00,2023-03-26
27,06055,0500000US06055,06,055,Napa,County,748.362,"POLYGON ((-122.10328 38.51335, -122.08884 38.3...",6055,Napa,California,1.0,2023-03-26 00:00:00,2023-03-26
28,06089,0500000US06089,06,089,Shasta,County,3775.402,"POLYGON ((-123.06543 40.28697, -123.06879 40.3...",6089,Shasta,California,1.0,2023-03-26 00:00:00,2023-03-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,54035,0500000US54035,54,035,Jackson,County,464.348,"POLYGON ((-81.52217 38.61275, -81.63284 38.554...",54035,Jackson,West Virginia,4355.0,2023-03-26 00:00:00,2023-03-26
3214,54041,0500000US54041,54,041,Lewis,County,384.895,"POLYGON ((-80.45733 38.73917, -80.44615 38.777...",54041,Lewis,West Virginia,684.0,2023-03-26 00:00:00,2023-03-26
3217,51021,0500000US51021,51,021,Bland,County,357.725,"POLYGON ((-81.22510 37.23487, -81.20477 37.243...",51021,Bland,Virginia,58.0,2023-03-26 00:00:00,2023-03-26
3218,51027,0500000US51027,51,027,Buchanan,County,502.763,"POLYGON ((-81.96830 37.53780, -81.92787 37.512...",51027,Buchanan,Virginia,4.0,2023-03-26 00:00:00,2023-03-26


In [68]:
counties_gdf

,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry,fips_code
0,01001,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",1001
1,01009,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",1009
2,01017,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413 32.87053, -85.12342 32.772...",1017
3,01021,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",1021
4,01033,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.13999 34.58170, -88.13925 34.587...",1033
...,...,...,...,...,...,...,...,...,...
3216,51001,0500000US51001,51,001,Accomack,County,449.496,"MULTIPOLYGON (((-75.24227 38.02721, -75.29687 ...",51001
3217,51021,0500000US51021,51,021,Bland,County,357.725,"POLYGON ((-81.22510 37.23487, -81.20477 37.243...",51021
3218,51027,0500000US51027,51,027,Buchanan,County,502.763,"POLYGON ((-81.96830 37.53780, -81.92787 37.512...",51027
3219,51037,0500000US51037,51,037,Charlotte,County,475.271,"POLYGON ((-78.44332 37.07940, -78.49303 36.891...",51037


In [114]:
time_step_merged

,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry,fips_code,county,state,sum,run_start_time,datetime
0,01001,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",1001,NaN,NaN,0.0,NaN,NaT
1,01009,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",1009,Blount,Alabama,4.0,2023-03-31 00:00:00,2023-03-31
2,01017,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413 32.87053, -85.12342 32.772...",1017,NaN,NaN,0.0,2023-03-31 00:00:00,2023-03-31
3,01021,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",1021,NaN,NaN,0.0,2023-03-31 00:00:00,2023-03-31
4,01033,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.13999 34.58170, -88.13925 34.587...",1033,NaN,NaN,0.0,2023-03-31 00:00:00,2023-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,51001,0500000US51001,51,001,Accomack,County,449.496,"MULTIPOLYGON (((-75.24227 38.02721, -75.29687 ...",51001,NaN,NaN,0.0,2023-03-31 00:00:00,2023-03-31
3217,51021,0500000US51021,51,021,Bland,County,357.725,"POLYGON ((-81.22510 37.23487, -81.20477 37.243...",51021,Bland,Virginia,4.0,2023-03-31 00:00:00,2023-03-31
3218,51027,0500000US51027,51,027,Buchanan,County,502.763,"POLYGON ((-81.96830 37.53780, -81.92787 37.512...",51027,NaN,NaN,0.0,2023-03-31 00:00:00,2023-03-31
3219,51037,0500000US51037,51,037,Charlotte,County,475.271,"POLYGON ((-78.44332 37.07940, -78.49303 36.891...",51037,NaN,NaN,0.0,2023-03-31 00:00:00,2023-03-31


In [98]:
filtered_data_copy = filtered_data.copy(deep=True)

In [168]:


# Visualize an outage event


# Create a list and directory to store frames
frame_files = []
dir_frames = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/figures/eaglei_frames"
os.makedirs(dir_frames, exist_ok=True)

# Loop through each datetime and update the color of each county
for i, time_step in enumerate(filtered_data['run_start_time'].unique()[0::4]):#:16]:

    time_step_df = filtered_data_copy[filtered_data_copy["datetime"] == pd.to_datetime(time_step)]

    time_step_merged = counties_gdf.merge(time_step_df, on='fips_code', how='left')
    cols_to_fill = ['run_start_time', 'datetime']
    time_step_merged[cols_to_fill] = time_step_merged[cols_to_fill].ffill()
    
    time_step_merged['run_start_time'].fillna(time_step, inplace=True)
    time_step_merged['datetime'].fillna(target_date, inplace=True)
    time_step_merged['sum'].fillna(0, inplace=True)


    # Create an animated choropleth map using Plotly Express
    fig = px.choropleth(time_step_merged,
                        geojson=time_step_merged.geometry,
                        locations=time_step_merged.index,
                        color='sum',
                        hover_name='county',
                        hover_data=['state', 'sum', 'datetime'],
                        # animation_frame='datetime',
                        color_continuous_scale="Reds",  # Color scale for the sum (customers affected)
                        range_color=(filtered_data["sum"].min(), filtered_data["sum"].max()/10.),  # Normalize color range
                        title=f"Power Grid Outages on {time_step}")
    
   

    # Update the map layout (focus on continental US)
    fig.update_geos(
        # fitbounds="locations",  # Ensure the map fits within the county locations
        visible=False,  # Hide default geos
        projection_type="albers usa",  # Use Albers projection for the US map
    )
    
    # Improve layout with margins and colorbar
    fig.update_layout(
        margin={"r": 0, "t": 50, "l": 0, "b": 0},  # Remove margins for a cleaner look
        coloraxis_colorbar=dict(title="Customers Affected"),  # Add color bar with title
        # title="Power Grid Outages Over Time",  # Add a plot title
        geo=dict(showcoastlines=True)
    )
    
    fig.update_traces(marker_line_color="rgba(0,0,0,0.3)", marker_line_width=0.5)

    # Save frame
    frame_filename = f"{dir_frames}/frame_{i:03d}.png"
    fig.write_image(frame_filename)
    frame_files.append(frame_filename)
    
    # # Show the plot
    # fig.show()
    
    # # Pause to simulate animation (adjust delay as needed)
    # time.sleep(1)



In [ ]:
frame_files

In [171]:
# Create a GIF or video
video_dir = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/figures/eaglei_movies"
os.makedirs(video_dir, exist_ok=True)

output_gif = os.path.join(video_dir,"power_outage_animation_March31_2023.gif")
output_mp4 = os.path.join(video_dir,"power_outage_animation_March31_2023.mp4")

# Create GIF
with imageio.get_writer(output_gif, mode="I", duration=1) as writer:
    for frame in frame_files:
        image = imageio.imread(frame)
        writer.append_data(image)


# Get all PNG files
frame_files = sorted(glob.glob(os.path.join(video_dir,"/*.png"))  

# Rename files to frame_000.png, frame_001.png, etc.
for i, old_name in enumerate(frame_files):
    new_name = f"frames/frame_{i:03d}.png"
    os.rename(old_name, new_name)

print("Renaming complete. Running ffmpeg...")

# Now run ffmpeg
os.system("ffmpeg -r 1 -i frames/frame_%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p power_outage_animation.mp4")

print("Video saved as power_outage_animation.mp4")
print(f"GIF saved as {output_gif}")
print(f"Video saved as {output_mp4}")

GIF saved as /Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/figures/eaglei_movies/power_outage_animation_March31_2023.gif
Video saved as /Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/figures/eaglei_movies/power_outage_animation_March31_2023.mp4


ffmpeg version N-112916-g3047f05a99-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-conf

In [184]:
frame_files = sorted(glob.glob(os.path.join(dir_frames,"/*.png")))


In [175]:
"{video_dir}/frame_{%03d}.png"
"ffmpeg -r 1 -i "+ video_dir/frame_%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p {output_mp4}"

'{video_dir}/frame_{%03d}.png'

In [185]:
frame_files

[]

In [187]:
os.system(f'ffmpeg -r 10 -pattern_type glob -i "{dir_frames}/*.png" -vcodec libx264 -crf 25 -pix_fmt yuv420p {video_dir}/power_outage_animation.mp4')




ffmpeg version N-112916-g3047f05a99-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-conf

0

In [188]:
f'ffmpeg -r 10 -pattern_type glob -i "{dir_frames}/*.png" -vcodec libx264 -crf 25 -pix_fmt yuv420p {video_dir}/power_outage_animation.mp4'

'ffmpeg -r 10 -pattern_type glob -i "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/figures/eaglei_frames/*.png" -vcodec libx264 -crf 25 -pix_fmt yuv420p /Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/figures/eaglei_movies/power_outage_animation.mp4'